In [1]:
from datasets import load_dataset

dataset = load_dataset("berkeley-nest/Nectar")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [50]:
import json
import regex as re
import random
from llm_blender.blender.blender_utils import get_pair_from_conv

random.seed(42)
new_data = []
num_pair_samples_per_instance = 2
for i, item in enumerate(dataset['train']):
    human_instructions = re.findall(r'(?<=Human:)(?:.|\n)*?(?=Assistant:)', item['prompt'])
    assistant_responses = re.findall(r'(?<=Assistant:)(?:.|\n)*?(?=Human:)', item['prompt'])
    human_instructions = [x.strip(' \n') for x in human_instructions]
    assistant_responses = [x.strip(' \n') for x in assistant_responses]
    assert len(human_instructions) == len(assistant_responses) + 1, f"Error in {i}"
    assert len(human_instructions) > 0, f"Error in {i}"
    
    if len(human_instructions) == 1:
        instruction = human_instructions[0]
        input = ""
    elif len(human_instructions) > 1:
        instruction = "Respond to the latest instruction based on the conversation history."
        input = item['prompt'].strip(' \n')
    candidate_texts = [x['answer'] for x in item['answers']]
    candiate_scores = [-x['rank'] for x in item['answers']]
    candidate_models = [x['model'] for x in item['answers']]
    candidates = [
        {
            'text': candidate_texts[j],
            'model': candidate_models[j],
            'decoding_method': 'unknown',
            'scores': {
                'human_preference': candiate_scores[j]
            }
        }
        for j in range(len(candidate_texts))
    ]
    random.shuffle(candidates)
    
    assert num_pair_samples_per_instance * 2 <= len(candidate_texts), f"Error in {i}"
    for j in range(0, num_pair_samples_per_instance * 2, 2):
        new_item = {
            'id': "berkeley-nest-nectar-" + str(i) + "-pair_" + str(j),
            'instruction': instruction,
            'input': input,
            'candidates': candidates[j*num_pair_samples_per_instance:(j+1)*num_pair_samples_per_instance]
        }
        new_data.append(new_item)
    
random.seed(42)
random_idx = random.sample(range(len(new_data)), 1000)
train_data = [new_data[i] for i in range(len(new_data)) if i not in random_idx]
val_data = [new_data[i] for i in range(len(new_data)) if i in random_idx]
with open("train_data_prepared.json", "w") as f:
    json.dump(train_data, f, indent=4)

with open("val_data_prepared.json", "w") as f:
    json.dump(val_data, f, indent=4)
    